In [1]:
import xml.etree.ElementTree as ET

In [2]:
centerline_tree = ET.parse('centerline.xml')
centerline_root = centerline_tree.getroot()
centerline_nodes= centerline_root.find('nodes')
centerline_links= centerline_root.find('links')

In [3]:
osm_tree = ET.parse('osm.xml')
osm_root = osm_tree.getroot()

In [4]:
way_to_link={}
for osm_links in osm_root.findall('links'):
    for osm_lk in osm_links.findall('link'):
        for osm_attrs in osm_lk.findall('attributes'):
            for osm_attr in osm_attrs.findall('attribute'):
                if (osm_attr.get('name')=="osm:way:id"):
                    way_to_link[osm_attr.text] = osm_lk.get('id')

In [5]:
#for each centerline 
for centerline_link in centerline_links.findall('link'):
    for osm in centerline_link.findall('osm'):
        osm_id = osm.get('osm_id').split(' ')
        for single_osm_id in osm_id:
            #for each osm id
            for osm_link in osm_links.findall('link'):
                if (single_osm_id in way_to_link.keys()):
                    if (osm_link.get('id') == way_to_link[single_osm_id]):
                        
                        for parametre in ['freespeed', 'capacity', 'permlanes','oneway','modes'] :
                            #union osmid info
                            osm_parametre = centerline_link.find(parametre)
                            if (osm_parametre!=None):
                                osm_parametre.set(parametre, osm_parametre.get(parametre)+" "+ osm_link.get(parametre))
                            else:
                                ET.SubElement(centerline_link, parametre,{parametre: osm_link.get(parametre)})
      

In [6]:
tree = ET.ElementTree(centerline_root)
ET.indent(tree)
tree.write('output.xml',encoding='utf-8')